In [20]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

OPENAI_API_BASE = os.environ.get("OPENAI_API_BASE")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")


model = ChatOpenAI(model="qwen-max")

In [3]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "C:\\Home\\Documents\\Projects\\models\\BAAI\\bge-large-zh-v1.5"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
embedding_model = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

# Indexing: Load

In [6]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(file_path="../data/对话马斯克.pdf")
docs = loader.load()

type(docs), type(docs[0])

(list, langchain_core.documents.base.Document)

In [8]:
print(docs[0])

page_content='472022 年第1期
对话埃隆·马斯克
——何时能在火星上看到人类？
钱颖一
人类距离登陆火星还有多
远？我们未来真的能够在火星上
生存吗？SpaceX首席执行官和
首席设计师埃隆·马斯克（Elon 
Musk）曾表示，到 2025 年有 50%
的概率可以在火星上看到人类。
就在 2022 年初，据美国《华尔街
日报》报道，埃隆·马斯克说，
如果飞行安全监管机构要求对公
司在德克萨斯州的一个设施进行
额外的环境评估，SpaceX准备在
佛罗里达州的一个场地来测试公
司一直在开发的巨型火箭。
据介绍，SpaceX 一直在准备
位于德克萨斯州南端 Boca Chica的一个场地，用于试飞星舰火箭
系统。230英尺高的超级重型助
推器将把星舰飞船送入轨道，该
公司曾表示，这将是其最雄心勃
勃的任务。  
马斯克表示，如果FAA（美
国联邦航空局）要求提交环境影
响报告，SpaceX公司将不得不将
星舰测试转移到肯尼迪航天中心。
他还补充道，随着时间的推移，
公司最终可能会将佛罗里达和德
克萨斯的设施用于不同用途：德
克萨斯州的工厂“非常适合作为
先进的研发基地”，肯尼迪航天
中心可能会成为“主要的运营发
射基地。”
早在七年前，他在与经济学
家、时任清华大学经济管理学院
院长的钱颖一对话时表示，在火
星上建立城市才是他创立SpaceX
的初衷和根本目的。
追随顶尖科技
钱颖一  我了解到 ，你早年从
南非到加拿大 ，进入女王大学
（Queen’s University），
两年之后转学到美国宾夕法尼亚
大学。在这两所大学你一开始学的都是商科，在宾大你又学了物
理学。 在大学学习商科和物理学，
你最大的收获是什么？大学教育
如何影响了你后来的职业生涯？
马斯克  我的大学教育并没有怎
么规划过。事实上，我当初都不
确定是否要上大学。
钱颖一  你是说，当初你甚至没
有想要上大学吗？
马斯克  是啊，我不确定是否该
上大学。当初我从南非到北美，
只是因为觉得很多尖端科技都是
在北美诞生、发展的，我了解到
的每一项新技术，似乎都是从那
里发源的。17 岁的时候，我独自
去了北美，也没有上大学的明确
计划，只是想接近技术的诞生地
而已。
钱颖一  必须是北美？钱颖一，著名经济学家。清华大
学 1977 级数学专业本科，曾任清
华大学经济管理学院院长

# Indexing: Split

In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)

all_splits = text_splitter.split_documents(docs)

len(all_splits)

13

In [11]:
len(all_splits[0].page_content)

991

In [12]:
all_splits[10].metadata

{'source': '../data/对话马斯克.pdf', 'page': 5, 'start_index': 787}

# Indexing: Store

In [13]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams


# Vector Store
client = QdrantClient(host="localhost", port=6333)

if not client.collection_exists("demo_collection"):
    client.create_collection(
        collection_name="demo_collection",
        vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
    )

vector_store = QdrantVectorStore(
    client=client,
    collection_name="demo_collection",
    embedding=embedding_model,
)

In [14]:
from uuid import uuid4

ids = [str(uuid4()) for _ in range(len(all_splits))]
vector_store.add_documents(documents=all_splits, ids=ids)

['3b05743e-66e6-4c60-a6e5-1a535dc6e741',
 '5bce9d80-e593-49d6-bb4d-4b95d4145d1c',
 '5caf2120-7086-49a0-a752-5d6512a697a2',
 'd9354cf1-e910-4486-acb6-b7d3a6f26ab1',
 '4a22a9b4-7ef3-4812-918c-f33528ba96d1',
 '7febc09c-9c3a-4c06-b815-af6b5d40bc77',
 '5135bbfd-4ec1-4eca-b69b-37700dc391a4',
 '671cebec-edb5-4561-94cc-8930226ca80d',
 'a14b061e-379a-4f0e-82b2-4a306c7bc584',
 '5afac4f2-4da5-41a7-9841-4faf4d55f0bd',
 '33c5ff02-0cb4-489b-b3a1-7e808c286db1',
 'd8a7b8bc-696b-450c-95f2-98a0272c6237',
 '9ef0e233-9352-44b8-a3a2-65eb7ba374a4']

# Retrieval and Generation: Retrieve

In [15]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

retrieved_docs = retriever.invoke("物理学对马斯克有什么影响？")

len(retrieved_docs)

3

# Retrieval and Generation: Generate

In [25]:
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "你是负责问答任务的助手。"
    "使用下面检索到的上下文来回答问题。"
    "如果你不知道答案，就说你不知道。"
    "最多使用三句话并保持答案简洁。"
    "上下文：\n\n{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [38]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain


question_answer_chain = create_stuff_documents_chain(llm=model, prompt=prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)
response = rag_chain.invoke({"input": "物理学对马斯克有什么影响？"})

In [39]:
import textwrap

print(textwrap.fill(response["answer"], 30))

物理学为马斯克提供了一个研究框架，尤其是通过第一性原理来理解
复杂和反直觉的事物。他认为这是了解新事物、探索未知领域的唯一
有效方法。此外，这种思维方式帮助他在创新时能够追溯到问题的本
源，而不是仅仅依赖类推。


In [41]:
for document in response["context"]:
    print(textwrap.fill(document.page_content, 70))
    print()

他们说还可以回去再读，不收学 费了。 钱颖一  你先学了商科，然后学 了物理。相比经济学和商科，你 认为物理学更有趣还是更难？ 马斯克
我之所以对物理和其他 科学感兴趣，是因为我想弄明白 现实世界的本质。生命的意义是什么？宇宙的本质是什么？这是 我读物理的主要动机。其实我学
商科的原因是我担心毕业后遇到 一个有商科学位的老板，如果老 板懂的我不懂，那我就没有优势 了。当然，其实也没什么。 钱颖一
你学商科是为了跟你未 来的老板有共同语言，这是当时 的动机吗？ 马斯克  是的，我最怕遇上一个 我不喜欢的老板。 钱颖一
然后你就决定自己当老 板，避免这个问题？ 马斯克  没错。 用物理学第一性原理思考 问题 钱颖一  你在很多场合都提到了 物理学
。你喜欢物理学 。你说 过物理学的第一性原理 （ F i r s t   Principles）能帮助你思考复杂
的事情。能否告诉我们，或举一 些例子，为什么物理学第一性原 理对你来说这么重要？对你的生 活和思维方式到底有什么影响？ 马斯克
要知道，想理解那些反 直觉的新事物，我觉得物理学提 供了一个最理想的研究框架。比 如说， 量子力学就是违背直觉的，
现实世界似乎并不是那样运转， 而事实就是，并可以通过实验高 度精确地验证。物理学之所以能

书？你最喜欢读什么方面的书？ 科幻类、哲学类、宗教、历史、 生物？和我们分享一下。 马斯克  我觉得历史方面的书很
有意思。可以从历史、人物传记 中学到很多经验教训。比如说， 他们面临的困难，他们如何克服 困难。我喜欢莎士比亚，我的最 爱之一。
我还喜欢本·富兰克林。 钱颖一  富兰克林是一个伟大的 发明家。 马斯克  除此之外，他还在正确 的时间做了正确的事情。 钱颖一
你认为自己也是在正确 2015 年 10 月 22 日，钱颖一与特斯拉首席执行官埃隆·马斯克（Elon Musk）（右） 在 2015
清华管理全球论坛上对话。

492022 年第1期 够在这些反直觉领域取得进展， 就是因为它将事物拆分到最基本 的实质，再往上推。我觉得这是
很重要的方法，事实上也是了解 新事物、探索未知领域的唯一有 效方法。 在日常生活中，我们非常善 用类推（analogy）方法，在别人
正在做的事情上做些微小调整。类 推提供了捷径，不需要大量思考， 这在日常生活中没什么问题。我 们不可能万事都用